In [7]:
import os as os
from pathlib import Path
import numpy as np
import pandas as pd
# import polars as pl
import tqdm
import slippi as slp
from joblib import Parallel, delayed
from multiprocessing import Manager
import pyarrow as pa
# import pyarrow.parquet as pq
import gzip
import pickle
import feather
import uuid
import sys
import time
sys.path.append('../..')
from slp.slp_package.slp_functions import one_hot_encode_flags

In [8]:
df = pd.read_parquet("C:\\Users\\jaspa\\Grant ML\\slp\\data\\all_game_data_df.parquet")

In [9]:
print(df.columns)
df.head()


Index(['path', 'length', 'is_teams', 'player_1_port',
       'player_1_character_name', 'player_1_type_name', 'player_1_stocks',
       'player_1_costume', 'player_1_team_value',
       'player_1_ucf_shield_drop_name', 'player_1_tag',
       'player_1_display_name', 'player_2_port', 'player_2_character_name',
       'player_2_type_name', 'player_2_stocks', 'player_2_costume',
       'player_2_team_value', 'player_2_ucf_shield_drop_name', 'player_2_tag',
       'player_2_display_name', 'random_seed', 'slippi', 'stage_name',
       'is_pal', 'is_frozen_ps', 'end_method_name', 'lras_initiator',
       'player_1_placement', 'player_2_placement', 'player_1_all_data_df',
       'player_1_pre_data_df', 'player_1_post_data_df',
       'player_1_input_data_df', 'player_2_all_data_df',
       'player_2_pre_data_df', 'player_2_post_data_df',
       'player_2_input_data_df', 'date', 'duration', 'platform',
       'player_1_netplay_code', 'player_1_netplay_name',
       'player_2_netplay_code', 'pl

,path,length,is_teams,player_1_port,player_1_character_name,player_1_type_name,player_1_stocks,player_1_costume,player_1_team_value,player_1_ucf_shield_drop_name,...,player_2_post_data_df,player_2_input_data_df,date,duration,platform,player_1_netplay_code,player_1_netplay_name,player_2_netplay_code,player_2_netplay_name,console_name
0,D:\ranked\ranked-anonymized-1-116248\ranked-an...,6092,False,0,FOX,HUMAN,4,2,None,UCF,...,C:\Users\jaspa\Grant ML\ranked_frame_data\CAPT...,C:\Users\jaspa\Grant ML\ranked_frame_data\CAPT...,NaT,NaN,None,None,None,None,None,None
1,D:\ranked\ranked-anonymized-1-116248\ranked-an...,6196,False,0,CAPTAIN_FALCON,HUMAN,4,0,None,UCF,...,C:\Users\jaspa\Grant ML\ranked_frame_data\GANO...,C:\Users\jaspa\Grant ML\ranked_frame_data\GANO...,NaT,NaN,None,None,None,None,None,None
2,D:\ranked\ranked-anonymized-1-116248\ranked-an...,6929,False,0,SHEIK,HUMAN,4,2,None,UCF,...,C:\Users\jaspa\Grant ML\ranked_frame_data\FOX\...,C:\Users\jaspa\Grant ML\ranked_frame_data\FOX\...,NaT,NaN,None,None,None,None,None,None
3,D:\ranked\ranked-anonymized-1-116248\ranked-an...,7112,False,0,CAPTAIN_FALCON,HUMAN,4,3,None,UCF,...,C:\Users\jaspa\Grant ML\ranked_frame_data\MART...,C:\Users\jaspa\Grant ML\ranked_frame_data\MART...,NaT,NaN,None,None,None,None,None,None
4,D:\ranked\ranked-anonymized-1-116248\ranked-an...,7001,False,0,FOX,HUMAN,4,0,None,UCF,...,C:\Users\jaspa\Grant ML\ranked_frame_data\JIGG...,C:\Users\jaspa\Grant ML\ranked_frame_data\JIGG...,NaT,NaN,None,None,None,None,None,None


In [10]:
df.describe()

,length,player_1_port,player_1_stocks,player_1_costume,player_2_port,player_2_stocks,player_2_costume,random_seed,end_method_name,lras_initiator,player_1_placement,player_2_placement,duration
count,1360.000000,1360.000000,1360.000000,1360.000000,1360.000000,1360.000000,1360.000000,1.360000e+03,1360.000000,225.000000,1360.0,1360.0,866.000000
mean,9306.044853,0.050735,4.000735,1.268382,1.365441,4.000735,1.331618,7.030413e+08,2.827206,0.497778,1.0,2.0,8206.243649
std,3600.175268,0.273289,0.027116,1.190173,0.719422,0.027116,1.291120,1.303407e+09,1.858575,0.714171,0.0,0.0,3177.286707
min,124.000000,0.000000,4.000000,0.000000,1.000000,4.000000,0.000000,1.000000e+01,2.000000,0.000000,1.0,2.0,124.000000
25%,7440.250000,0.000000,4.000000,0.000000,1.000000,4.000000,0.000000,2.548650e+04,2.000000,0.000000,1.0,2.0,6828.000000
50%,9296.500000,0.000000,4.000000,1.000000,1.000000,4.000000,1.000000,4.607600e+04,2.000000,0.000000,1.0,2.0,8653.000000
75%,11234.250000,0.000000,4.000000,2.000000,1.000000,4.000000,2.000000,9.443625e+08,2.000000,1.000000,1.0,2.0,10202.750000
max,25947.000000,2.000000,5.000000,5.000000,3.000000,5.000000,5.000000,4.289742e+09,7.000000,3.000000,1.0,2.0,17329.000000


In [11]:
path_list = df['player_1_pre_data_df'].to_list() + df['player_2_pre_data_df'].to_list()
print(path_list)

['C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\FOX\\f465beda-42a7-4a3e-81bb-d2986da5e311.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\CAPTAIN_FALCON\\879981ec-9ead-4b87-b058-8290dde45a8b.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\SHEIK\\f7d7279e-9e2e-4596-859f-d1ba2486167c.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\CAPTAIN_FALCON\\30917957-e162-45e1-8474-7c0eb805c9b8.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\FOX\\b8780a24-e3a8-428f-aa4f-6388fee1d441.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\FOX\\08ab5802-70cf-49d1-ac28-b46e42d907ca.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\SHEIK\\db123c24-1a70-4ae5-ac98-64d1d3389960.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\GAME_AND_WATCH\\849691f9-2be1-4aa4-a672-d571b3fd48dc.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\FOX\\961de9b2-323c-40f5-9f0e-6af48451caf2.parquet', 'C:\\Users\\jaspa\\Grant ML\\ranked_frame_data\\FALCO\\b865537

In [12]:
frame_data_df = pd.read_parquet(path_list[0])

In [13]:
frame_data_df.columns
# frame_data_df['pre_buttons_physical'][2]

Index(['frame_index', 'pre_state', 'pre_position_x', 'pre_position_y',
       'pre_direction', 'pre_joystick_x', 'pre_joystick_y', 'pre_cstick_x',
       'pre_cstick_y', 'pre_triggers_logical', 'pre_triggers_physical_l',
       'pre_triggers_physical_r', 'pre_buttons_logical',
       'pre_buttons_physical', 'pre_random_seed', 'pre_raw_analog_x',
       'pre_damage'],
      dtype='object')

In [14]:
def create_directories(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [15]:
# def process_file(path,time_1,time_2,time_3):
#     save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    
#     time_1_start = time.time()
#     frame_data_df = pd.read_feather(path)
#     time_1.append(time.time()-time_1_start)
    
#     time_2_start = time.time()
#     inputs = np.empty((4,frame_data_df.shape[0]),dtype = float)
#     for i in range(frame_data_df.shape[0]):
#         inputs[0, i] = frame_data_df['pre_joystick_x'][i]
#         inputs[1, i] = frame_data_df['pre_joystick_y'][i]
#         inputs[2, i] = frame_data_df['pre_cstick_x'][i]
#         inputs[3, i] = frame_data_df['pre_cstick_y'][i] 
#         # inputs[4:,i] = one_hot_encode_flags(slp.event.Buttons.Physical,frame_data_df['pre_buttons_physical'][i])
#     time_2.append(time.time()-time_2_start)
    
#     time_3_start = time.time()
#     full_path = os.path.join(save_path, str(uuid.uuid4()) + '.npy.gz')
#     with gzip.open(full_path, 'wb') as f:
#         np.save(f, inputs)
#     time_3.append(time.time() - time_3_start)
#     return inputs         


def process_file(path,time_1,time_2,time_3,buttons):
    save_path = "C:\\Users\\jaspa\\Grant ML\\feather df frame date test"
    time_1_start = time.time()
    frame_data_df = pd.read_parquet(path,engine='pyarrow')
    time_1.append(time.time()-time_1_start)
    num_rows = frame_data_df.shape[0]

    # Assuming one_hot_encode_flags can process a series and return a 2D array
    # encoded_flags = one_hot_encode_flags(slp.event.Buttons.Physical, frame_data_df['pre_buttons_physical'])


    time_2_start = time.time()

    # button_states = frame_data_df['pre_buttons_logical'].to_numpy()[:, None]
    # bitmasks = np.array([button.value for button in buttons if button != buttons.NONE])[None, :]
    # encoded = (button_states & bitmasks) != 0
    # inputs = np.empty((num_rows,25), dtype=float)
    # inputs[ :,0] = frame_data_df['pre_joystick_x']
    # inputs[ :,1] = frame_data_df['pre_joystick_y']
    # inputs[ :,2] = frame_data_df['pre_cstick_x']
    # inputs[ :,2] = frame_data_df['pre_cstick_y']
    # inputs[:,4:] = encoded
    # inputs.T
    
    # Convert the 'pre_buttons_logical' column to a NumPy array
    button_states = frame_data_df['pre_buttons_logical'].values

    # For each button, apply the bitmask and add a column to the DataFrame
    for button in buttons:
        if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
            continue
        # Use broadcasting to apply the bitmask and generate the one-hot encoded array
    frame_data_df[button.name] = (button_states & button.value) != 0
    
    time_2.append(time.time()-time_2_start)
    
    time_3_start = time.time()
    # full_path = os.path.join(save_path, f"{uuid.uuid4()}.npy.gz")
    # with gzip.open(full_path, 'wb') as f:
    #     np.save(f, inputs)
    time_3.append(time.time() - time_3_start)
    
    return 

In [16]:
# process_file(path_list[0],1)
manager = Manager()
time_1 = manager.list()
time_2 = manager.list()
time_3 = manager.list()

buttons = slp.event.Buttons.Logical
        
# Use joblib to parallelize processing of SLP files
Parallel(n_jobs=-1, verbose=1)(delayed(process_file)(path,time_1,time_2,time_3,buttons) for path in tqdm.tqdm(path_list))



100%|██████████| 2720/2720 [00:03<00:00, 778.97it/s] 
[Parallel(n_jobs=-1)]: Done 2720 out of 2720 | elapsed:    3.7s finished


[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,

In [17]:
print(sum(time_1))
print(sum(time_2))
print(sum(time_3))


35.26237154006958
1.1560215950012207
0.0


In [18]:
# frame_data_df =   pd.read_parquet(path_list[1],engine='pyarrow')

# button_states = frame_data_df['pre_buttons_logical'].values

# # For each button, apply the bitmask and add a column to the DataFrame
# for button in buttons:
#     if button == buttons.NONE:  # Skip the NONE value to avoid an unnecessary column
#         continue
#     # Use broadcasting to apply the bitmask and generate the one-hot enco
#     # ed array
#     frame_data_df[button.name] = (button_states & button.value) != 0

In [19]:
# frame_data_df.columns